In [1]:
import sys
sys.path.insert(0, "../src")
sys.path.insert(0, "..")
import src
import torch
import matplotlib.pyplot as plt

np.set_printoptions(precision=2)
torch.cuda.empty_cache()

This piece of code runs a grid search over layers and states to generate the right panel of Figure #3 (mnist result)
The funciton to generate it is defined as:  
def run_part_lin(name, start, end, gap , allstates ,deviceId, num_devices, reports = [100,200,400,800,1600], N = 2048):

* name- the file where the results will be saved to
* start - first layer to start the grid from
* end - last leayer to run the grid on
* gap - how much layers to jump over for the grid
* allstates - the states that will appear on the X axis
* deviceId - cuda device to use
* num devices - number of devices working  on this job (to balance load between them)
* reports - report test accuracy and training accuracy after how many steps?
* N- width of the fully connected layers

This grid search takes a long time to complete (~ day when distributed).  
Using the commented-out code will generate the graph presented in the article.  
The uncommented code will generate a more simple heatmap with very little details.
To distribute the code, run a similar command with different cur_gpu and total_gpus values (This will not work from single jupyter notebook)

In [ ]:

first_layer = 10
last_layer = 135
layers_skip = 25
cur_gpu = 0
total_gpus = 1
reports = [100,200,400]

##allstates = [3,4,5,6,7,8,9,10,11,12,13,14,15,16]
allstates = [4,7,10]
##mnist.run_part_lin("grid_all_v3", first_layer, last_layer, layers_skip, allstates , cur_gpu, total_gpus)
src.mnist.run_part_lin("example_states", first_layer, last_layer, layers_skip, allstates , cur_gpu, total_gpus, reports = reports, N = 256)


Generate the heatmaps and more:

In [ ]:
import matplotlib.pyplot as plt

results={}
if (1):
    ##aggregate the distributed results to a single objects
    name = "example_states"
    for i in range(4): ## 4 - maximum number of GPUs supported.
        try:
            results.update(mnist.load_obj("%s_%d"% (name,i)))
        except:
            continue
    mnist.save_obj(results,name)
    
if (1):
    fig = plt.figure()
    mnist.showgrid(name=name,steps=100,test=1, fig = fig , subplot=131)
    mnist.showgrid(name=name,steps=200,test=1, fig = fig , subplot=132)
    mnist.showgrid(name=name,steps=400,test=1, fig = fig , subplot=133)
    plt.show()

if (1):
    fig = plt.figure()
    mnist.showContour(name=name,steps=100,test=1, fig = fig , subplot=131, levels=3)
    mnist.showContour(name=name,steps=200,test=1, fig = fig , subplot=132, levels=3)
    mnist.showContour(name=name,steps=400,test=1, fig = fig , subplot=133, levels=3)
    plt.gcf().set_size_inches(15, 5)

    plt.show()

